<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/VPN_%C4%B0LE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from binance.client import Client
from ta.trend import MACD
import time

# Binance API key ve secret bilgilerinizi buraya girin
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'

# Binance client'ı oluştur
client = Client(api_key, api_secret)

# Tüm coinleri çek
print("Binance'teki tüm coinler çekiliyor...")
exchange_info = client.get_exchange_info()
symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['status'] == 'TRADING']
print(f"Toplam {len(symbols)} coin bulundu.")

# MACD hesaplama fonksiyonu
def calculate_macd(symbol, interval='1h', short_window=12, long_window=26, signal_window=9):
    try:
        # Binance'tan mum verilerini çek
        klines = client.get_klines(symbol=symbol, interval=interval, limit=100)

        # Verileri DataFrame'e dönüştür
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
            'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
            'taker_buy_quote_asset_volume', 'ignore'
        ])

        # 'close' sütununu sayısal değere dönüştür
        df['close'] = pd.to_numeric(df['close'])

        # MACD hesapla
        macd = MACD(df['close'], window_slow=long_window, window_fast=short_window, window_sign=signal_window)
        df['macd'] = macd.macd()
        df['macd_signal'] = macd.macd_signal()
        df['macd_diff'] = macd.macd_diff()

        # Son değeri döndür
        return df.iloc[-1]
    except Exception as e:
        print(f"{symbol} için MACD hesaplanırken hata oluştu: {e}")
        return None

# Alım sinyali veren coinleri bul
def find_buy_signals():
    buy_signals = []
    print("MACD alım sinyalleri aranıyor...")

    for symbol in symbols:
        try:
            macd_data = calculate_macd(symbol)
            if macd_data is not None:
                if macd_data['macd'] > macd_data['macd_signal'] and macd_data['macd_diff'] > 0:
                    buy_signals.append(symbol)
                    print(f"Alım Sinyali: {symbol} - MACD: {macd_data['macd']:.4f}, MACD Signal: {macd_data['macd_signal']:.4f}")
        except Exception as e:
            print(f"{symbol} işlenirken hata oluştu: {e}")

        # API rate limitini aşmamak için bekleme süresi
        time.sleep(0.1)

    return buy_signals

# Alım sinyali veren coinleri listele
buy_signals = find_buy_signals()
print("\nAlım Sinyali Veren Coinler:")
for coin in buy_signals:
    print(coin)